In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 10)


AttributeError: module 'numpy' has no attribute 'round'

In [5]:
df = pd.read_csv("data.csv")
df

,FUNDOS,TIPO,PREÇO ATUAL (R$),LIQUIDEZ DIÁRIA (R$),P/VP,DIVIDEND YIELD,DY (12M) ACUMULADO,DY (3M) MÉDIA,DY (12M) MÉDIA,VARIAÇÃO PREÇO,...,VPA,P/VPA,DY PATRIMONIAL,VARIAÇÃO PATRIMONIAL,QUANT. ATIVOS,VOLATILIDADE,NUM. COTISTAS,TAX. GESTÃO,TAX. PERFORMANCE,TAX. ADMINISTRAÇÃO
0,PRTS11,SHOPPING,"117,70",NaN,"1.961,67",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,NaN,11.000,NaN,NaN,NaN
1,SRVD11,TIJOLO,"3,00","233,91","583,00","2,29 %","3,30 %","1,65 %","1,65 %","-9,79 %",...,"1,19","2,53","16,87 %","-0,11 %",0,"135,21",107.000,NaN,NaN,NaN
2,IBFF11,TIJOLO,"66,40","162.549,50","4,49","0,00 %","8,02 %","0,15 %","0,67 %","0,00 %",...,"14,78","4,49","100,08 %","-79,09 %",0,"4,20",3.870,NaN,NaN,NaN
3,RBRM11,DESENVOLVIMENTO,"94.794,81",NaN,"3,45","0,00 %","0,00 %","0,00 %","0,00 %","0,00 %",...,NaN,NaN,NaN,NaN,0,NaN,54.000,NaN,NaN,NaN
4,LOFT11B,DESENVOLVIMENTO,"29,90",NaN,"2,84","0,00 %","189,83 %","0,00 %","15,82 %","0,00 %",...,NaN,NaN,NaN,NaN,16,NaN,1.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,?,?,"42,00",1.027,"0,60","1,57 %","15,34 %","1,06 %","1,28 %","-7,79 %",...,NaN,NaN,NaN,NaN,0,"67,71",373.000,NaN,NaN,NaN
426,?,?,"9,74","4.527.574,42","1,01","1,20 %","15,22 %","1,25 %","1,27 %","2,68 %",...,"9,64","1,01","1,24 %","0,00 %",0,"10,43",226.183,NaN,NaN,NaN
427,?,?,"103,00","213.783,30","1,09","0,39 %","0,39 %","0,39 %","0,39 %","0,00 %",...,NaN,NaN,NaN,NaN,1,"85,27",286.000,NaN,NaN,NaN
428,?,?,"110,72","15.546.428,15","0,99","0,76 %","9,50 %","0,76 %","0,71 %","-1,06 %",...,"112,31","0,99","0,76 %","11,26 %",2,"13,99",370.163,NaN,NaN,NaN


In [6]:

# Dropando as 3 ultimas colunas
colunas_NA = ["TAX. GESTÃO", "TAX. PERFORMANCE", "TAX. ADMINISTRAÇÃO"]
df.drop(colunas_NA, axis=1, inplace=True)

print(df)

      FUNDOS             TIPO PREÇO ATUAL (R$) LIQUIDEZ DIÁRIA (R$)      P/VP  \
0     PRTS11         SHOPPING           117,70                  NaN  1.961,67   
1     SRVD11           TIJOLO             3,00               233,91    583,00   
2     IBFF11           TIJOLO            66,40           162.549,50      4,49   
3     RBRM11  DESENVOLVIMENTO        94.794,81                  NaN      3,45   
4    LOFT11B  DESENVOLVIMENTO            29,90                  NaN      2,84   
..       ...              ...              ...                  ...       ...   
425        ?                ?            42,00                1.027      0,60   
426        ?                ?             9,74         4.527.574,42      1,01   
427        ?                ?           103,00           213.783,30      1,09   
428        ?                ?           110,72        15.546.428,15      0,99   
429        ?                ?           105,05                  NaN      0,83   

    DIVIDEND YIELD DY (12M)

In [7]:
# Renomenado colunas para conterem o simbolo % no final
porcentagem = ["DIVIDEND YIELD", "DY (12M) ACUMULADO","DY (3M) MÉDIA", "DY (12M) MÉDIA", "VARIAÇÃO PREÇO", "DY PATRIMONIAL", "VARIAÇÃO PATRIMONIAL"]
for coluna in porcentagem:
    df[coluna] = df[coluna].str.strip(" %")
    str = coluna + " (%)"
    df = df.rename(columns={coluna: str})
df

,FUNDOS,TIPO,PREÇO ATUAL (R$),LIQUIDEZ DIÁRIA (R$),P/VP,DIVIDEND YIELD (%),DY (12M) ACUMULADO (%),DY (3M) MÉDIA (%),DY (12M) MÉDIA (%),VARIAÇÃO PREÇO (%),PATRIMÔNIO LÍQUIDO,VPA,P/VPA,DY PATRIMONIAL (%),VARIAÇÃO PATRIMONIAL (%),QUANT. ATIVOS,VOLATILIDADE,NUM. COTISTAS
0,PRTS11,SHOPPING,"117,70",NaN,"1.961,67",NaN,NaN,NaN,NaN,NaN,"3.111.607,44",NaN,NaN,NaN,NaN,2,NaN,11.000
1,SRVD11,TIJOLO,"3,00","233,91","583,00","2,29","3,30","1,65","1,65","-9,79","-720.474,07","1,19","2,53","16,87","-0,11",0,"135,21",107.000
2,IBFF11,TIJOLO,"66,40","162.549,50","4,49","0,00","8,02","0,15","0,67","0,00","9.855.905,16","14,78","4,49","100,08","-79,09",0,"4,20",3.870
3,RBRM11,DESENVOLVIMENTO,"94.794,81",NaN,"3,45","0,00","0,00","0,00","0,00","0,00","14.823.845,13",NaN,NaN,NaN,NaN,0,NaN,54.000
4,LOFT11B,DESENVOLVIMENTO,"29,90",NaN,"2,84","0,00","189,83","0,00","15,82","0,00","20.644.665,73",NaN,NaN,NaN,NaN,16,NaN,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,?,?,"42,00",1.027,"0,60","1,57","15,34","1,06","1,28","-7,79",39.778.168,NaN,NaN,NaN,NaN,0,"67,71",373.000
426,?,?,"9,74","4.527.574,42","1,01","1,20","15,22","1,25","1,27","2,68",994.767.385,"9,64","1,01","1,24","0,00",0,"10,43",226.183
427,?,?,"103,00","213.783,30","1,09","0,39","0,39","0,39","0,39","0,00",167.038.269,NaN,NaN,NaN,NaN,1,"85,27",286.000
428,?,?,"110,72","15.546.428,15","0,99","0,76","9,50","0,76","0,71","-1,06",3.461.722.290,"112,31","0,99","0,76","11,26",2,"13,99",370.163


In [8]:
# alterando os valores que estavam em string para float, alterando o separador de milhar das colunas porcentuais
numerais =["PREÇO ATUAL (R$)","LIQUIDEZ DIÁRIA (R$)", "P/VP","DIVIDEND YIELD (%)","DY (12M) ACUMULADO (%)","DY (3M) MÉDIA (%)","DY (12M) MÉDIA (%)", "VARIAÇÃO PREÇO (%)", "PATRIMÔNIO LÍQUIDO", "VPA","P/VPA", "DY PATRIMONIAL (%)","VARIAÇÃO PATRIMONIAL (%)", "VOLATILIDADE"]

for coluna in numerais:
    df[coluna] = df[coluna].str.replace(".", "")
    df[coluna] = df[coluna].str.replace(",", ".")
    df[coluna] = df[coluna].astype(float)

#df.drop(["DIVIDEND YIELD (%)", "DY (12M) ACUMULADO (%)", "DY (3M) MÉDIA (%)", "DY PATRIMONIAL (%)", "TAX. GESTÃO", "TAX. PERFORMANCE", "TAX. ADMINISTRAÇÃO"], axis=1, inplace=True)

df

,FUNDOS,TIPO,PREÇO ATUAL (R$),LIQUIDEZ DIÁRIA (R$),P/VP,DIVIDEND YIELD (%),DY (12M) ACUMULADO (%),DY (3M) MÉDIA (%),DY (12M) MÉDIA (%),VARIAÇÃO PREÇO (%),PATRIMÔNIO LÍQUIDO,VPA,P/VPA,DY PATRIMONIAL (%),VARIAÇÃO PATRIMONIAL (%),QUANT. ATIVOS,VOLATILIDADE,NUM. COTISTAS
0,PRTS11,SHOPPING,117.70,NaN,1961.67,NaN,NaN,NaN,NaN,NaN,3.111607e+06,NaN,NaN,NaN,NaN,2,NaN,11.000
1,SRVD11,TIJOLO,3.00,233.91,583.00,2.29,3.30,1.65,1.65,-9.79,-7.204741e+05,1.19,2.53,16.87,-0.11,0,135.21,107.000
2,IBFF11,TIJOLO,66.40,162549.50,4.49,0.00,8.02,0.15,0.67,0.00,9.855905e+06,14.78,4.49,100.08,-79.09,0,4.20,3.870
3,RBRM11,DESENVOLVIMENTO,94794.81,NaN,3.45,0.00,0.00,0.00,0.00,0.00,1.482385e+07,NaN,NaN,NaN,NaN,0,NaN,54.000
4,LOFT11B,DESENVOLVIMENTO,29.90,NaN,2.84,0.00,189.83,0.00,15.82,0.00,2.064467e+07,NaN,NaN,NaN,NaN,16,NaN,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,?,?,42.00,1027.00,0.60,1.57,15.34,1.06,1.28,-7.79,3.977817e+07,NaN,NaN,NaN,NaN,0,67.71,373.000
426,?,?,9.74,4527574.42,1.01,1.20,15.22,1.25,1.27,2.68,9.947674e+08,9.64,1.01,1.24,0.00,0,10.43,226.183
427,?,?,103.00,213783.30,1.09,0.39,0.39,0.39,0.39,0.00,1.670383e+08,NaN,NaN,NaN,NaN,1,85.27,286.000
428,?,?,110.72,15546428.15,0.99,0.76,9.50,0.76,0.71,-1.06,3.461722e+09,112.31,0.99,0.76,11.26,2,13.99,370.163


In [9]:
# verificando a matriz de correlação
colunas = ["PREÇO ATUAL (R$)", "LIQUIDEZ DIÁRIA (R$)", "P/VP", "DIVIDEND YIELD (%)", "DY (12M) ACUMULADO (%)", "DY (3M) MÉDIA (%)", "DY (12M) MÉDIA (%)", "VARIAÇÃO PREÇO (%)", "PATRIMÔNIO LÍQUIDO", "VPA", "P/VPA","VARIAÇÃO PATRIMONIAL (%)", "DY PATRIMONIAL (%)",	"QUANT. ATIVOS", "VOLATILIDADE", "NUM. COTISTAS"]
matriz_cor = df[colunas].corr()

plt.figure(figsize=(12,8))
sns.heatmap(matriz_cor, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Matriz Correlacao")
plt.show

NameError: name 'plt' is not defined

In [20]:
#newDf = df[["TIPO","LIQUIDEZ DIÁRIA (R$)","P/VP", "DY (12M) MÉDIA (%)", "PATRIMÔNIO LÍQUIDO", "VPA", "QUANT. ATIVOS", "NUM. COTISTAS"]].copy()

# sem o patrimonio liquido nem liquidez diaria eem vpa pq excluiria os ?

newDf = df[["TIPO","P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]].copy()

#newDf.dropna(how='any', axis=0, inplace = True)

# Supondo que 'df' seja o seu DataFrame e você tenha 'grupos' e 'colunas' como listas de nomes dos grupos e colunas, respectivamente
grupos =["TIJOLO","PAPEL","MISTO","DESENVOLVIMENTO","SHOPPING"]

#for grupo in grupos:
    #newDf[newDf['TIPO'] == grupo] = newDf[newDf['TIPO'] == grupo].fillna(newDf[newDf['TIPO'] == grupo].mean())

colunas = ["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]

# Criar um novo DataFrame para armazenar os newDf


# Preencher valores NaN nas colunas específicas usando a média de cada grupo
for coluna in colunas:
    for grupo in grupos:
        # Calcula a média do grupo na coluna atual e preenche os NaN na coluna correspondente
        media_grupo = newDf.loc[newDf['TIPO'] == grupo, coluna].mean()
        newDf.loc[newDf['TIPO'] == grupo, coluna] = newDf.loc[newDf['TIPO'] == grupo, coluna].fillna(media_grupo)













#newDf["NUM. COTISTAS"] = newDf["NUM. COTISTAS"].apply(lambda x: int(x * 1000))
pd.set_option('display.max_rows', 10)

newDf

#retirada de outliners
#newDf = newDf[newDf["P/VP"] <= 5]
#newDf = newDf[newDf["DY (12M) MÉDIA (%)"] <= 20]

#print(newDf)

,TIPO,P/VP,DY (12M) MÉDIA (%),QUANT. ATIVOS,NUM. COTISTAS
0,SHOPPING,1961.67,0.610476,2,11.000
1,TIJOLO,583.00,1.650000,0,107.000
2,TIJOLO,4.49,0.670000,0,3.870
3,DESENVOLVIMENTO,3.45,0.000000,0,54.000
4,DESENVOLVIMENTO,2.84,15.820000,16,1.000
...,...,...,...,...,...
425,?,0.60,1.280000,0,373.000
426,?,1.01,1.270000,0,226.183
427,?,1.09,0.390000,1,286.000
428,?,0.99,0.710000,2,370.163


In [22]:
y = newDf["TIPO"]
# Verificando o balanceamento das classes
print("Classes pré balanceamento:", Counter(y))

# Balanceando as classes

# Filtrando as amostras da classe Tijolo
dfPredict = newDf[newDf["TIPO"] != "?"]

print(dfPredict)

x = dfPredict[["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]]
y = dfPredict["TIPO"]

# Convertendo DataFrames para matrizes NumPy
x = x.values
y = y.values

print(int(len(y[y == "TIJOLO"]) / 2))

# Reduzindo pela metade a quantidade de amostras da classe Tijolo (undersampling)
rus = RandomUnderSampler(sampling_strategy={"TIJOLO": int(len(y[y == "TIJOLO"]) / 2)})
x_resampled, y_resampled = rus.fit_resample(x, y)

# Verificando a contagem de classes após o balanceamento
print("Contagem de classes após o balanceamento:", Counter(y_resampled))

NameError: name 'Counter' is not defined

In [ ]:
# dfPredict = newDf[newDf["TIPO"] != "?"]

x = dfPredict[["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]]
y = dfPredict["TIPO"]
#  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=100)

clf = DecisionTreeClassifier()

# Definindo os hiperparâmetros a serem testados
param_grid = {
    'max_depth': [None, 3,4,5, 10, 15],       # (representando a profundidade máxima da árvore) 
    'min_samples_leaf': [2, 3, 4],      # Número mínimo de amostras necessárias para ser uma folha (nó final)
    'min_samples_split': [2, 3, 4, 5]   # Número mínimo de amostras necessárias para dividir um nó interno.
}

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Treinando o modelo com diferentes combinações de hiperparâmetros
grid_search.fit(x_train, y_train)

# Obtendo os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

# Avaliando o modelo com os melhores hiperparâmetros
best_clf = DecisionTreeClassifier(**best_params)
best_clf.fit(x_train, y_train)
y_pred = best_clf.predict(x_test)

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia com os melhores hiperparâmetros:", accuracy)

newData = newDf[newDf["TIPO"] == "?"]
newData = newData[["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]]
print(newData)

predict = best_clf.predict(newData)
print(predict)




'''
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

newData = newDf[newDf["TIPO"] == "?"]
newData = newData[["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]]
print(newData)

predict = clf.predict(newData)
print(predict)
'''

#tijolo , papel, tijolo, shopping, tijolo INICIAL
# Tijolo, Papel, Tijolo, Tijolo, Tijolo

# papel, papel, shooping, tijolo, tijolo
# papel, papel, desenvolvimento, tijolo, tijolo



#plt.figure(figsize=(20, 10))
#plot_tree(clf, filled=True, feature_names=x.columns, class_names=dfPredict["TIPO"].unique(), rounded=True)
#plt.savefig('decision_tree.png')  # Adjust the filename and format as needed
#plt.show()

Melhores hiperparâmetros: {'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 3}
Acurácia com os melhores hiperparâmetros: 0.4489795918367347
     P/VP  DY (12M) MÉDIA (%)  QUANT. ATIVOS  NUM. COTISTAS
425 0.600               1.280              0         373000
426 1.010               1.270              0         226183
427 1.090               0.390              1         286000
428 0.990               0.710              2         370163
429 0.830               0.650              4         220000
['PAPEL' 'PAPEL' 'TIJOLO' 'TIJOLO' 'TIJOLO']


c:\Users\Ramon Godoy Izidoro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


'\nclf.fit(x_train, y_train)\n\ny_pred = clf.predict(x_test)\n\naccuracy = accuracy_score(y_test, y_pred)\nprint(accuracy)\n\nnewData = newDf[newDf["TIPO"] == "?"]\nnewData = newData[["P/VP", "DY (12M) MÉDIA (%)", "QUANT. ATIVOS", "NUM. COTISTAS"]]\nprint(newData)\n\npredict = clf.predict(newData)\nprint(predict)\n'